# Assignment 6: Hidden Markov Models

## Submission
All submissions will be via Gradescope. If you're completing this assignment in Jupyter Notebook, you must run the `notebook2script.py` file to export your work to a python file. To generate your submission file, run the command 

`python notebook2script.py submission`

and your file will be created under the `submission` directory.

Upload the resulting `submission.py` file to the Assignment 6 assignment on Gradescope for feedback.

#### IMPORTANT: A total of 10 submissions is allowed for this assignment. Please use your submissions carefully and do not submit until you have thoroughly tested your code locally.

#### If you're at 9 submissions, use your tenth and last submission wisely. The submission marked as ‘Active’ in Gradescope will be the submission counted towards your grade. 

Please also submit your submission.py to Canvas as backup.



## Overview

The goal of this assignment is to demonstrate the power of probabilistic models. You will build a word recognizer for American Sign Language (ASL) video sequences. In particular, this project employs hidden Markov models (HMM's) to analyze a series of measurements taken from videos of isolated American Sign Language (ASL) signs collected for research. (see the [Isolated Sign Language Recognition Corpus](https://www.kaggle.com/competitions/asl-signs/)).

In each video, an ASL signer signs a meaningful sentence. In a typical ASL recognition system, you observe the XY coordinates of the speaker's left hand, right hand, and nose for every frame. The following diagram shows how the positions of the left hand (Red), right hand (Blue), and nose (Green) change over time. Saturation of colors represents time elapsed:

![](demo/hands_nose_position.png) 

In this assignment, for the sake of simplicity, you will only use the Y-coordinates of the right hand and the right thumb to construct your HMM. In Part 1 you will build a one dimensional model, recognizing words based only on a series of right-hand Y coordinates; in Part 2 you will go multidimensional and utilize both the right hand and the right thumb features. At this point, you will have two observed coordinates at each time step (frame) representing right hand & right thumb Y positions.

The words you will be recognizing are "ALLIGATOR", "NUTS", and "SLEEP". These individual signs can be seen in the sign phrases from our dataset:

![](demo/alligator-singlesign-00000015.gif) 

### ALLIGATOR 

![](demo/nuts-singlesign-00000016.gif) 

### NUTS 

![](demo/sleep-singlesign-00000001.gif) 

### SLEEP 

### Part 1a: Encoding the HMM
### _[CS6601: 15 Points]_ _[CS3600: 25 Points]_

Follow the method taught on Udacity **Lecture 8: 29. HMM Training** to determine following values for each word:
1. the transition probabilities of each state
2. the mean & standard deviation of emission Gaussian distribution of each state

Use the training samples from the table below. Provide the transition, prior, and emission probabilities parameters for all three words with **accuracy to 3 decimal digits**.

Round the values to 3 decimal places thoughout entire assignment:
- 0.1 stays 0.1 or 0.100
- 0.1234 rounds to 0.123
- 0.2345 rounds to 0.235
- 0.3456 rounds to 0.346
- 0.0123 rounds to 0.012
- 0.0125 rounds to 0.013

#### When calculating the self transition probability (e.g. P(A2 -> A2)), you should calculate it as 1 - exit transition probability (e.g. 1 - P(A2->A3))

Those values can be hardcoded in your program. Don't use round() from python.

Word | Frames | Observed sequence | Initial State1 | Initial State2 | Initial State3
--- | --- | --- | --- | --- | --- 
ALLIGATOR | 17 | 31, 28, 28, 37, 68, 49, 64, 66, 22, 17, 53, 73, 81, 78, 48, 49, 47 | 31, 28, 28, 37, 68, 49 | 64, 66, 22, 17, 53, 73 | 81, 78, 48, 49, 47
ALLIGATOR | 10 | 25, 62, 75, 80, 75, 36, 74, 33, 27, 34 | 25, 62, 75, 80 | 75, 36, 74, 33 | 27, 34
ALLIGATOR | 16 | -4, 69, 59, 45, 62, 22, 17, 28, 12, 14, 24, 32, 39, 61, 35, 32 | -4, 69, 59, 45, 62, 22 | 17, 28, 12, 14, 24, 32 | 39, 61, 35, 32
NUTS | 11 | 45, 68, 62, 75, 61, 44, 73, 72, 71, 75, 55 | 45, 68, 62, 75 | 61, 44, 73, 72 | 71, 75, 55
NUTS | 18 | 33, 33, 32, 32, 34, 38, 43, 41, 35, 36, 36, 37, 38, 38, 39, 40, 38, 38 | 33, 33, 32, 32, 34, 38 | 43, 41, 35, 36, 36, 37 | 38, 38, 39, 40, 38, 38
NUTS | 19 | 33, 31, 29, 28, 25, 24, 25, 28, 28, 38, 37, 40, 37, 36, 36, 38, 44, 48, 48 | 33, 31, 29, 28, 25, 24, 25 | 28, 28, 38, 37, 40, 37, 36 | 36, 38, 44, 48, 48
SLEEP | 8 | 37, 35, 41, 39, 41, 38, 38, 38 | 37, 35, 41 | 39, 41, 38 | 38, 38
SLEEP | 12 | 22, 17, 18, 35, 33, 36, 42, 36, 41, 41, 37, 38 | 22, 17, 18, 35 | 33, 36, 42, 36 | 41, 41, 37, 38
SLEEP | 13 | 38, 37, 35, 32, 35, 13, 36, 41, 41, 31, 32, 34, 34 | 38, 37, 35, 32, 35 | 13, 36, 41, 41, 31 | 32, 34, 34

As shown in the diagram below, each one of the three words (ALLIGATOR, NUTS, and SLEEP) has exactly **THREE hidden states** in its HMM. All words have equal probability of occuring. Modify the prior accordingly. All words must start from State 1 and can only transit to the next state or stay in the current one as shown below:

<img src="part_1_a_probs.png">

### _Training sequences need to have 3 hidden states no matter what!_
If you follow the procedure on the Udacity lecture video, you might encounter a situation where a hidden state is **_squeezed_** out by an adjacent state. In that situation, always keep at least one observation for that hidden state.

Example:
Assume you've reached a stage where the following is true: 
- State 1 has mean=53 & std=6
- State 2 has mean=37 & std=9
- State 3 has mean=70 & std=8

The next training sample has the following observed sequence:

`45 45 34 | 30 30 25 36 52 | 62 69 74` 

and you are trying to adjust the location of state boundary between State 1 & 2. You first move it 1 step to the left since 34 is closer to State 2, and then you realize that 45 is still closer to State 2. If you follow the same routine, you will end up with no obvervation for State 1. In order to prevent this from happening, you have to stop at the last "45" and as a result leave the boundary as 

`45 | 45 34 30 30 25 36 52 | 62 69 74`

Now you meet the '3 hidden states per sample' requirement.

### Some hints/guidelines for training:
#### 1. How should we compare if an observation if closer to one state or another?
Check how many standard deviations away is the observation from the mean for each state. 
Example: Say 46 is the rightmost observation in State 1. If we denote the mean and std of State i as μi,σi, then should we be comparing 
|46−μ1| / σ1 vs |46−μ2| / σ2

#### 2. For HMM training, which side of the boundary should we check first while assigning observed sequence values to states?
After computing the mean and std for each state, adjust the boundary between the states. Always start from the 1st element at the LEFT side of the boundary. If the LEFT element is closer to the next state, then move the boundary leftward. If the LEFT element should stay at the current state, then check the RIGHT element. This is just done to make sure that everyone gets the same results in the context of the assignment.

In [19]:
import hmm_submission_test as tests

In [20]:
#export
import numpy as np
import operator

Load my Data

In [21]:
# def my_data():
a1 = np.array([31, 28, 28, 37, 68, 49, 64, 66, 22, 17, 53, 73, 81, 78, 48, 49, 47],dtype=np.int8)
a2 = np.array([25, 62, 75, 80, 75, 36, 74, 33, 27, 34],dtype=np.int8)
a3 = np.array([-4, 69, 59, 45, 62, 22, 17, 28, 12, 14, 24, 32, 39, 61, 35, 32],dtype=np.int8)
n1 = np.array([45, 68, 62, 75, 61, 44, 73, 72, 71, 75, 55],dtype=np.int8)
n2 = np.array([33, 33, 32, 32, 34, 38, 43, 41, 35, 36, 36, 37, 38, 38, 39, 40, 38, 38],dtype=np.int8)
n3 = np.array([33, 31, 29, 28, 25, 24, 25, 28, 28, 38, 37, 40, 37, 36, 36, 38, 44, 48, 48],dtype=np.int8)
s1 = np.array([37, 35, 41, 39, 41, 38, 38, 38],dtype=np.int8)
s2 = np.array([22, 17, 18, 35, 33, 36, 42, 36, 41, 41, 37, 38],dtype=np.int8)
s3 = np.array([38, 37, 35, 32, 35, 13, 36, 41, 41, 31, 32, 34, 34],dtype=np.int8)
a_combined = np.array((a1,a2,a3), dtype=object)
n_combined = np.array((n1,n2,n3), dtype=object)
s_combined = np.array((s1,s2,s3), dtype=object)



In [22]:
def show_partitions(sample,p1,p2):
    # print (f"😀sample = {sample}")
    return  f"{sample[:p1]} 🟢 {sample[p1:p2]} 🟢 {sample[p2:]}"
def total_stddev(sample,p1,p2):
    assert 0 < p1 < p2 < len(sample), f"total_stddev(): partitions must start strictly inside the array. Condition fail (0 < {p1} < {p2} < {len(sample)})"
    return np.std(sample[:p1]) + np.std(sample[p1:p2]) + np.std(sample[p2:])

def improve_partition(sample: np.ndarray, p1:int,p2:int, mu_left:float,mu_middle:float,
                      mu_right:float,stddev_left:float,
                      stddev_middle:float,stddev_right:float):
    p1_old, p2_old = p1, p2

    assert p1 is not None and p2 is not None, "partitions should be valid"

    sample_length = sample.shape[0]
    print (f"sample: {sample}, partitions = ({p1},{p2}), len = {sample_length}")
    assert 0 < p1 < p2 < sample_length, f"improve_partition(): partitions must start strictly inside the array. Condition fail (0 < {p1} < {p2} < {sample_length})"

    # for k in range(20):
    #     print (f"improve_partition(): left iteration = {k}")
        # sample [p1] is the first element of the next group 
    p1_moved = False
    #check if element to left of the border p1 wants to move right (by moving the barrier left)
    while abs(sample[p1-1] - mu_left) / stddev_left > abs(sample[p1-1] - mu_middle)/stddev_middle  and  1 < p1:
        print ("   1st partition moves ⬅️ ", show_partitions(sample,p1,p2), \
                f" left_fit={round(abs(sample[p1-1] - mu_left) / stddev_left,3)},middle_fit={round(abs(sample[p1-1] - mu_middle)/stddev_middle,3)},")
        print(f"   improvement= {total_stddev(sample,p1-1,p2)<total_stddev(sample,p1,p2)}")
        p1 -= 1
        p1_moved = True
        #check if element to right of the border p1 wants to move left (by moving the barrier right)
    while  abs(sample[p1] - mu_left) / stddev_left < abs(sample[p1] - mu_middle)/stddev_middle   and p1 < p2-1 and not p1_moved :
        print ("   1st partition moves ➡️ ", show_partitions(sample,p1,p2), \
                f" left_fit={round(abs(sample[p1] - mu_left) / stddev_left ,3)},middle_fit={round(abs(sample[p1] - mu_middle)/stddev_middle,3)},")
        print(f"   improvement= {total_stddev(sample,p1+1,p2)<total_stddev(sample,p1,p2)}")
        p1 += 1
    # else:
        # print ("p1: ended without reaching end of loop 😀")
            # break
        # if k==19:
        #     print (f"p1: reached end of loop 🚩🚩🚩")
    # for k in range(20):
    #     print (f"improve_partition(): right iteration = {k}")
    p2_moved = False
    while abs(sample[p2-1] - mu_middle)/stddev_middle > abs(sample[p2-1] - mu_right)/stddev_right and p1 < p2-1:
        print ("   2nd partition moves ⬅️ ",  show_partitions(sample,p1,p2), \
                f" middle_fit={round(abs(sample[p2-1] - mu_middle)/stddev_middle,3)},right_fit={round(abs(sample[p2-1] - mu_right)/stddev_right,3)},")
        print(f"   improvement= {total_stddev(sample,p1,p2-1)<total_stddev(sample,p1,p2)}")
        p2 -= 1
        p2_moved = True
    #check if element to right of the border p2 wants to move left (by moving the barrier right)
    while abs(sample[p2] - mu_middle)/stddev_middle  < abs(sample[p2] - mu_right)/stddev_right and  p2 <sample_length-1 and not p2_moved:
        print ("   2nd partition moves ➡️ ",show_partitions(sample,p1,p2) , \
                f" middle_fit={round(abs(sample[p2] - mu_middle)/stddev_middle,3)},right_fit={round(abs(sample[p2] - mu_right)/stddev_right ,3)},")
        print(f"   improvement= {total_stddev(sample,p1,p2+1)<total_stddev(sample,p1,p2)}")
        p2 += 1
    #check if element to left of the border p2 wants to move right (by moving the barrier left)
    # else:
    #     print ("p2: ended without reaching end of loop 😀")
        # break
        # if k==19:
        #     print (f"p2: reached end of loop 🚩🚩🚩")
    assert 0 < p1 < p2 < sample_length, f"improve_partition():  paritions must end strictly inside the array. Condition fail (0 < {p1} < {p2} < {sample_length})"

    partitions_moved = (p1_old == p1) and (p2_old == p2) #true if one of the paritions moved
    return (p1, p2, partitions_moved)
    

def partition_word(word_samples,initial_partition):
    partition = np.array(initial_partition,np.int8)
    print (f"partitions: \n{partition}",)
    for k in range (100): #1000 is an arbitrary limit to make sure we don't get stuck in infinite loop
        left_data = np.concatenate((word_samples[0][:partition[0,0]], \
                                    word_samples[1][:partition[1,0]], \
                                    word_samples[2][:partition[2,0]]))
        middle_data = np.concatenate((word_samples[0][partition[0,0]:partition[0,1]], \
                                      word_samples[1][partition[1,0]:partition[1,1]], \
                                      word_samples[2][partition[2,0]:partition[2,1]]))
        right_data = np.concatenate((word_samples[0][partition[0,1]:], \
                                     word_samples[1][partition[1,1]:], \
                                     word_samples[2][partition[2,1]:]))
        
        mu_left = np.mean(left_data)
        mu_middle = np.mean(middle_data)
        mu_right =  np.mean(right_data)
        stddev_left =  np.std(left_data)
        stddev_middle = np.std(middle_data)
        stddev_right =  np.std(right_data)
        partitions_old = partition.copy()
        print ("updated means, stddev 🐲, iteration k={k}")
        for i in range (3):
            print(f"🦕 word i={i}")
            (partition[i,0], partition[i,1], partitions_unchanged) = improve_partition(sample=word_samples[i], \
                                                            p1=partition[i,0], p2=partition[i,1], \
                                                            mu_left=mu_left,mu_middle=mu_middle, \
                                                            mu_right=mu_right,stddev_left=stddev_left, \
                                                            stddev_middle=stddev_middle,stddev_right=stddev_right)
        partitions_unchanged = np.array_equal(partitions_old, partition)
        if partitions_unchanged:
            print (f"no partitions moved, breaking (iteration = {k})")
            break
        else:
            print (f"partitions moved, continuing (iteration = {k})")

    assert left_data is not None   , "partition_word(): something is wrong left_data"
    assert middle_data is not None , "partition_word(): something is wrong middle_data"
    assert right_data is not None  , "partition_word(): something is wrong right_data"
    print ("done ⭐⭐⭐⭐⭐⭐⭐. Partition Follows:")
    print (partition)
    # print (f"left_data = {left_data}, middle_data = {middle_data}, right_data = {right_data}")

    left_data = np.concatenate((word_samples[0][:partition[0,0]], \
                            word_samples[1][:partition[1,0]], \
                            word_samples[2][:partition[2,0]]))
    middle_data = np.concatenate((word_samples[0][partition[0,0]:partition[0,1]], \
                                word_samples[1][partition[1,0]:partition[1,1]], \
                                word_samples[2][partition[2,0]:partition[2,1]]))
    right_data = np.concatenate((word_samples[0][partition[0,1]:], \
                                word_samples[1][partition[1,1]:], \
                                word_samples[2][partition[2,1]:]))

    mu_left = np.mean(left_data)
    mu_middle = np.mean(middle_data)
    mu_right =  np.mean(right_data)
    stddev_left =  np.std(left_data)
    stddev_middle = np.std(middle_data)
    stddev_right =  np.std(right_data)

    print(f"mu_left={mu_left} ,stddev_left={stddev_left}\nmu_middle={mu_middle} ,stddev_middle={stddev_middle} \n,mu_right={mu_right},stddev_right={stddev_right}  ")
    print(f"left frame count = {len(left_data)}, transition prob = {3.0/len(left_data)}, self = {1-3.0/len(left_data)}")
    print(f"middle frame count = {len(middle_data)}, transition prob = {3.0/len(middle_data)}, self = {1-3.0/len(middle_data)}")
    print(f"right frame count = {len(right_data)}, transition prob = {3.0/len(right_data)}, self = {1-3.0/len(right_data)}")
    return (left_data,middle_data,right_data)
    



In [23]:
a_partition = [[6,12],[4,8],[6,12]]
# left,middle,right = np.empty((1)),np.empty((1)),np.empty((1))
left,middle,right = partition_word(a_combined,initial_partition=a_partition)

partitions: 
[[ 6 12]
 [ 4  8]
 [ 6 12]]
updated means, stddev 🐲, iteration k={k}
🦕 word i=0
sample: [31 28 28 37 68 49 64 66 22 17 53 73 81 78 48 49 47], partitions = (6,12), len = 17
   1st partition moves ➡️  [31 28 28 37 68 49] 🟢 [64 66 22 17 53 73] 🟢 [81 78 48 49 47]  left_fit=0.796,middle_fit=1.056,
   improvement= False
   1st partition moves ➡️  [31 28 28 37 68 49 64] 🟢 [66 22 17 53 73] 🟢 [81 78 48 49 47]  left_fit=0.885,middle_fit=1.144,
   improvement= False
   2nd partition moves ⬅️  [31 28 28 37 68 49 64 66] 🟢 [22 17 53 73] 🟢 [81 78 48 49 47]  middle_fit=1.452,right_fit=1.427,
   improvement= True
   2nd partition moves ⬅️  [31 28 28 37 68 49 64 66] 🟢 [22 17 53] 🟢 [73 81 78 48 49 47]  middle_fit=0.572,right_fit=0.273,
   improvement= True
🦕 word i=1
sample: [25 62 75 80 75 36 74 33 27 34], partitions = (4,8), len = 10
   1st partition moves ➡️  [25 62 75 80] 🟢 [75 36 74 33] 🟢 [27 34]  left_fit=1.283,middle_fit=1.54,
   improvement= True
   2nd partition moves ➡️  [25 62 75 

In [24]:
n_partition = [[4,8],[6,12],[7,14]]

left,middle,right = partition_word(n_combined,initial_partition=n_partition)


partitions: 
[[ 4  8]
 [ 6 12]
 [ 7 14]]
updated means, stddev 🐲, iteration k={k}
🦕 word i=0
sample: [45 68 62 75 61 44 73 72 71 75 55], partitions = (4,8), len = 11
   2nd partition moves ⬅️  [45 68 62 75] 🟢 [61 44 73 72] 🟢 [71 75 55]  middle_fit=2.273,right_fit=2.13,
   improvement= True
   2nd partition moves ⬅️  [45 68 62 75] 🟢 [61 44 73] 🟢 [72 71 75 55]  middle_fit=2.35,right_fit=2.212,
   improvement= True
🦕 word i=1
sample: [33 33 32 32 34 38 43 41 35 36 36 37 38 38 39 40 38 38], partitions = (6,12), len = 18
   2nd partition moves ➡️  [33 33 32 32 34 38] 🟢 [43 41 35 36 36 37] 🟢 [38 38 39 40 38 38]  middle_fit=0.344,right_fit=0.671,
   improvement= True
   2nd partition moves ➡️  [33 33 32 32 34 38] 🟢 [43 41 35 36 36 37 38] 🟢 [38 39 40 38 38]  middle_fit=0.344,right_fit=0.671,
   improvement= True
   2nd partition moves ➡️  [33 33 32 32 34 38] 🟢 [43 41 35 36 36 37 38 38] 🟢 [39 40 38 38]  middle_fit=0.267,right_fit=0.588,
   improvement= True
   2nd partition moves ➡️  [33 33 32 

In [25]:
s_partition = [[3,6],[4,8],[5,10]]

left,middle,right = partition_word(s_combined,initial_partition=s_partition)


partitions: 
[[ 3  6]
 [ 4  8]
 [ 5 10]]
updated means, stddev 🐲, iteration k={k}
🦕 word i=0
sample: [37 35 41 39 41 38 38 38], partitions = (3,6), len = 8
   1st partition moves ⬅️  [37 35 41] 🟢 [39 41 38] 🟢 [38 38]  left_fit=1.18,middle_fit=0.717,
   improvement= True
   1st partition moves ⬅️  [37 35] 🟢 [41 39 41 38] 🟢 [38 38]  left_fit=0.408,middle_fit=0.077,
   improvement= True
   2nd partition moves ➡️  [37] 🟢 [35 41 39 41 38] 🟢 [38 38]  middle_fit=0.32,right_fit=0.34,
   improvement= True
🦕 word i=1
sample: [22 17 18 35 33 36 42 36 41 41 37 38], partitions = (4,8), len = 12
   1st partition moves ⬅️  [22 17 18 35] 🟢 [33 36 42 36] 🟢 [41 41 37 38]  left_fit=0.408,middle_fit=0.077,
   improvement= True
   2nd partition moves ➡️  [22 17 18] 🟢 [35 33 36 42 36] 🟢 [41 41 37 38]  middle_fit=0.717,right_fit=1.359,
   improvement= False
   2nd partition moves ➡️  [22 17 18] 🟢 [35 33 36 42 36 41] 🟢 [41 37 38]  middle_fit=0.717,right_fit=1.359,
   improvement= True
🦕 word i=2
sample: [38 3

In [26]:
#export
def part_1_a():
    """Provide probabilities for the word HMMs outlined below.
    Word ALLIGATOR, NUTS, and SLEEP.
    Review Udacity Lesson 8 - Video #29. HMM Training
    Returns:
        tuple() of
        (prior probabilities for all states for word ALLIGATOR,
         transition probabilities between states for word ALLIGATOR,
         emission parameters tuple(mean, std) for all states for word ALLIGATOR,
         prior probabilities for all states for word NUTS,
         transition probabilities between states for word NUTS,
         emission parameters tuple(mean, std) for all states for word NUTS,
         prior probabilities for all states for word SLEEP,
         transition probabilities between states for word SLEEP,
         emission parameters tuple(mean, std) for all states for word SLEEP)
        Sample Format (not complete):
        (
            {'A1': prob_of_starting_in_A1, 'A2': prob_of_starting_in_A2, ...},
            {'A1': {'A1': prob_of_transition_from_A1_to_A1,
                    'A2': prob_of_transition_from_A1_to_A2,
                    'A3': prob_of_transition_from_A1_to_A3,
                    'Aend': prob_of_transition_from_A1_to_Aend},
             'A2': {...}, ...},
            {'A1': tuple(mean_of_A1, standard_deviation_of_A1),
             'A2': tuple(mean_of_A2, standard_deviation_of_A2), ...},
            {'N1': prob_of_starting_in_N1, 'N2': prob_of_starting_in_N2, ...},
            {'N1': {'N1': prob_of_transition_from_N1_to_N1,
                    'N2': prob_of_transition_from_N1_to_N2,
                    'N3': prob_of_transition_from_N1_to_N3,
                    'Nend': prob_of_transition_from_N1_to_Nend},
             'N2': {...}, ...}
            {'N1': tuple(mean_of_N1, standard_deviation_of_N1),
             'N2': tuple(mean_of_N2, standard_deviation_of_N2), ...},
            {'S1': prob_of_starting_in_S1, 'S2': prob_of_starting_in_S2, ...},
            {'S1': {'S1': prob_of_transition_from_S1_to_S1,
                    'S2': prob_of_transition_from_S1_to_S2,
                    'S3': prob_of_transition_from_S1_to_S3,
                    'Send': prob_of_transition_from_S1_to_Send},
             'S2': {...}, ...}
            {'S1': tuple(mean_of_S1, standard_deviation_of_S1),
             'S2': tuple(mean_of_S2, standard_deviation_of_S2), ...} 
        )
    """


    """Word ALLIGATOR"""
    a_prior_probs = {
        'A1': 0.333,
        'A2': 0.,
        'A3': 0.,
        'Aend': 0.
    }
    a_transition_probs = {
        'A1': {'A2': 0.167, 'A3': 0., 'A1': 0.833, 'Aend': 0.},
        'A2': {'A1': 0., 'A2': 0.786, 'A3': 0.214, 'Aend': 0.},
        'A3': {'A2': 0., 'A1': 0., 'A3': 0.727, 'Aend': 0.273},
        'Aend': {'A2': 0., 'A3': 0., 'A1': 0., 'Aend': 1}
    }
    # Parameters for end state is not required͏󠄂͏️͏󠄌͏󠄎͏︆͏︋͏󠄏
    a_emission_paras = {
        'A1': (51.056, 21.986),
        'A2': (28.357, 14.936),
        'A3': (53.727, 16.707),
        'Aend': (None, None)
    }


#     mu_left=38.08108108108108 ,stddev_left=11.175209964478636
# mu_middle=42.0 ,stddev_middle=2.8284271247461903 
# ,mu_right=60.0,stddev_right=13.490737563232042  
# left frame count = 37, transition prob = 0.08108108108108109, self = 0.9189189189189189
# middle frame count = 3, transition prob = 1.0, self = 0.0
# right frame count = 8, transition prob = 0.375, self = 0.625
    """Word NUTS"""

    n_prior_probs = {
        'N1': 0.333,
        'N2': 0.,
        'N3': 0.,
        'Nend': 0.
    }
    # Probability of a state changing to another state.͏󠄂͏️͏󠄌͏󠄎͏︆͏︋͏󠄏
    n_transition_probs = {
        'N1': {'N3': 0., 'N1': 0.919, 'N2': 0.081, 'Nend': 0.},
        'N2': {'N3': 1.0, 'N1': 0., 'N2': 0., 'Nend': 0.},
        'N3': {'N3': 0.375, 'N2': 0., 'N1': 0., 'Nend': 0.625},
        'Nend': {'N1': 0., 'N2': 0., 'N3': 0., 'Nend': 1.0}
    }
    # Parameters for end state is not required͏󠄂͏️͏󠄌͏󠄎͏︆͏︋͏󠄏
    n_emission_paras = {
        'N1': (38.081, 11.175),
        'N2': (42.0, 2.828),
        'N3': (60.0, 13.491),
        'Nend': (None, None)
    }

# mu_left=29.5 ,stddev_left=8.411301920630361
# mu_middle=36.18181818181818 ,stddev_middle=5.989660512737866 
# ,mu_right=36.666666666666664,stddev_right=1.8856180831641267  
# left frame count = 8, transition prob = 0.375, self = 0.625
# middle frame count = 22, transition prob = 0.13636363636363635, self = 0.8636363636363636
# right frame count = 3, transition prob = 1.0, self = 0.0
    """Word SLEEP"""
    s_prior_probs = {
        'S1': 0.333,
        'S2': 0.,
        'S3': 0.,
        'Send': 0.
    }
    s_transition_probs = {
        'S1': {'S1': 0.625, 'S3': 0.,    'S2': 0.375, 'Send': 0.},
        'S2': {'S1': 0.,    'S2': 0.864, 'S3': 0.136, 'Send': 0.},
        'S3': {'S1': 0.,    'S3': 1.,    'S2': 0.,    'Send': 0.},
        'Send': {'S3': 0.0, 'S2': 0.,    'S1': 0.,    'Send': 1.}
    }
    # Parameters for end state is not required͏󠄂͏️͏󠄌͏󠄎͏︆͏︋͏󠄏
    s_emission_paras = {
        'S1': (29.5, 8.411),
        'S2': (36.182, 5.990),
        'S3': (36.667, 1.886),
        'Send': (None, None)
    }

    return (a_prior_probs, a_transition_probs, a_emission_paras,
            n_prior_probs, n_transition_probs, n_emission_paras,
            s_prior_probs, s_transition_probs, s_emission_paras)

########## DON'T WRITE ANY CODE OUTSIDE THE FUNCTION! ################͏󠄂͏️͏󠄌͏󠄎͏︆͏︋͏󠄏
##### CODE BELOW IS USED FOR RUNNING LOCAL TEST DON'T MODIFY IT ######͏󠄂͏️͏󠄌͏󠄎͏︆͏︋͏󠄏
tests.TestPart1a().test_prior(part_1_a)
tests.TestPart1a().test_a_emission(part_1_a)
tests.TestPart1a().test_a_transition(part_1_a)
tests.TestPart1a().test_n_emission(part_1_a)
tests.TestPart1a().test_n_transition(part_1_a)

tests.TestPart1a().test_s_emission(part_1_a)
tests.TestPart1a().test_s_transition(part_1_a)
################ END OF LOCAL TEST CODE SECTION ######################͏󠄂͏️͏󠄌͏󠄎͏︆͏︋͏󠄏

UnitTest test_prior passed successfully!
UnitTest test_a_emission passed successfully!
UnitTest test_a_transition passed successfully!
UnitTest test_n_emission passed successfully!
UnitTest test_n_transition passed successfully!
UnitTest test_s_emission passed successfully!
UnitTest test_s_transition passed successfully!


### Part 1b: Creating the Viterbi Trellis
### _[CS6601: 40 Points]_ _[CS3600: 75 Points]_

The goal here will be to use the HMM derived from Part 1a (states, prior probabilities, transition probabilities, and parameters of emission distribution) to build a viterbi trellis.  When provided with an evidence vector (list of observed right-hand Y coordinates), the function will return the most likely sequence of states that generated the evidence and the probabilty of that sequence being correct.

For example, an evidence vector [38, 37, 35, 32, 35, 13, 36, 41, 41, 31, 32, 34, 34] (last training sequence for "SLEEP") should output a sequence ['S1', ... 'S2', ... 'S3']

If no sequence can be found, the algorithm should return one of the following tuples:
`(None, 0)` (null),  `([], 0)` (empty list) or  `(['A1', 'A1', ... 'A1'],0)` (Or all being the first state of that letter)

"No sequence can be found" means the probability reaches 0 midway. If you find an incomplete sequence with some probability, output that sequence with its probability. 

Note: **DO NOT** consult any external sources other than the Wikipedia PDF in the assignment. Failure to abide by this requirement will lead to a 0 on the assignment.

#### Functions to complete:
1. `viterbi()`

#### Hint:
In order to reconstruct your most-likely path after running Viterbi, you'll need to keep track of a back-pointer at each state, which directs you to that state's most-likely predecessor.

You are asked to use the provided function `gaussian_prob` to compute  emission probabilities. Although in real work, you have to convert the probability to log-base in order to prevent digit underflow, in this assignment, we will only test your function against a rather short sequence of observations, so **DO NOT** convert the probability to logarithmic probability, otherwise you will fail the tests on Gradescope.

In the autograder, we will also test your code against other `evidence_vectors`.

In [27]:
#export
def gaussian_prob(x, para_tuple):
    """Compute the probability of a given x value

    Args:
        x (float): observation value
        para_tuple (tuple): contains two elements, (mean, standard deviation)

    Return:
        Probability of seeing a value "x" in a Gaussian distribution.

    Note:
        We simplify the problem so you don't have to take care of integrals.
        Theoretically speaking, the returned value is not a probability of x,
        since the probability of any single value x from a continuous 
        distribution should be zero, instead of the number outputted here.
        By definition, the Gaussian percentile of a given value "x"
        is computed based on the "area" under the curve, from left-most to x. 
        The probability of getting value "x" is zero because a single value "x"
        has zero width, however, the probability of a range of value can be
        computed, for say, from "x - 0.1" to "x + 0.1".

    """
    if list(para_tuple) == [None, None]:
        return 0.0

    mean, std = para_tuple
    gaussian_percentile = (2 * np.pi * std**2)**-0.5 * \
                          np.exp(-(x - mean)**2 / (2 * std**2))
    return gaussian_percentile



In [28]:
#export
    
    
def viterbi(evidence_vector, states, prior_probs,
            transition_probs, emission_paras):
    """Viterbi Algorithm to calculate the most likely states give the evidence.
    Args:
        evidence_vector (list): List of right hand Y-axis positions (integer).
        states (list): List of all states in a word. No transition between words.
                       example: ['A1', 'A2', 'A3', 'Aend', 'N1', 'N2', 'N3', 'Nend']
        prior_probs (dict): prior distribution for each state.
                            example: {'X1': 0.25,
                                      'X2': 0.25,
                                      'X3': 0.25,
                                      'Xend': 0.25}
        transition_probs (dict): dictionary representing transitions from each
                                 state to every other valid state such as for the above 
                                 states, there won't be a transition from 'A1' to 'N1'
        emission_paras (dict): parameters of Gaussian distribution 
                                from each state.
    Return:
        tuple of
        ( A list of states the most likely explains the evidence,
          probability this state sequence fits the evidence as a float )
    Note:
        You are required to use the function gaussian_prob to compute the
        emission probabilities.
    """
    print (f"evidence_vector={evidence_vector} \nstates={states}\nprior_probs={prior_probs}\ntransition_probs={transition_probs}\nemission_paras={emission_paras}")
    
    # TODO: complete this function.͏󠄂͏️͏󠄌͏󠄎͏︆͏︋͏󠄏
    if len(evidence_vector)==0:
        sequence = []
        probability = 0.0
        return sequence, probability

    if len(evidence_vector)==1:
        mymatrix = np.array([prior_probs["A1"] * gaussian_prob(evidence_vector[0], emission_paras["A1"]),
                             prior_probs["N1"] * gaussian_prob(evidence_vector[0], emission_paras["N1"]),
                             prior_probs["S1"] * gaussian_prob(evidence_vector[0], emission_paras["S1"])])
        probability = np.max(mymatrix)
        sequence = [np.array(["A1","N1","S1"])[np.argmax(mymatrix)]]
        return sequence, probability
    
    evidence_count = len(evidence_vector)
    state_count = len(states)
    assert isinstance(evidence_count,int), "evidence_count should be an int"
    assert isinstance(state_count   ,int), "state_count should be an int"
    
    probability = np.zeros([state_count  ,evidence_count],dtype=float) #  States (i) x Evidence Matrix (j). 
    parent = np.empty([state_count  ,evidence_count],dtype=int) # will store how we arrived 
    parent_string = np.empty([state_count  ,evidence_count],dtype=object) # will store how we arrived 

    #this matrix will hold the maximum probability to arrive at state
    
    
    # for i in range (state_count):
    #     for j in range (evidence_count):
    #         proability[i,j] = gaussian_prob(x=evidence_vector[j], para_tuple=emission_paras[states[i]])
        
    # print (f"prob_mat=\n{probability}")
    # prob_mat2 = np.fromfunction(lambda i, j: gaussian_prob(x=evidence_vector[j], para_tuple=emission_paras[states[i]]),\
    #                                                        shape=[state_count  ,evidence_count],\
    #                                                        dtype=np.float16) #  States x Evidence Matrix 
    # print (f"allclose= {np.allclose(prob_mat,prob_mat2)}")
    # we will go horizontally over the matrix, calculating for each step the probablilty of reaching that step
    # path =[str]
    for i in range (state_count):
        probability[i,0] = prior_probs[states[i]] * gaussian_prob(evidence_vector[0], emission_paras[states[i]])
        # parent[i,0] = state_to_idx
        # parent_string[state_from_idx,0] = states[i]
        # path[0] = states[np.argmax(probability[:,0])]
    
    for frame_number in range (1,evidence_count):
        for state_from_idx in range(state_count):
            for state_to_idx in range(state_from_idx,state_count):
                state_from,state_to=  states[state_from_idx],states[state_to_idx]
                if not state_to in transition_probs[state_from]:
                    # print (f"frame={frame_number} {state_from}➜{state_to}: skipping because no connection")
                    break
                if 0 == transition_probs[state_from][state_to]:
                    # print (f"frame={frame_number} {state_from}➜{state_to}: skipping because chance is zero")
                    break #if zero probability == no need to consider this one
                # current probablity is the chance of getting here via the path we got here, which is a multiplication of how we got here 
                current_probability = probability[state_from_idx,frame_number-1] * \
                                      transition_probs[state_from][state_to] * \
                                      gaussian_prob(evidence_vector[frame_number], emission_paras[state_to])
                # print (f"⭐new_prob {current_probability}= {probability[state_from_idx,frame_number-1]} * {transition_probs[state_from][state_to]} * {gaussian_prob(evidence_vector[frame_number], emission_paras[state_to])}")

                if current_probability > probability[state_to_idx,frame_number] :
                    #if we are here, then a new maximum occured and the new parent is ??
                    # print (f"frame={frame_number} {state_from}➜{state_to}: 💾 Updating: new prob={current_probability} > {probability[state_to_idx,frame_number]} New Parent: {states[state_to_idx]}")
                    probability[state_to_idx,frame_number] = current_probability
                    parent[state_to_idx,frame_number] = state_from_idx
                    parent_string[state_to_idx,frame_number] = states[state_from_idx]
                    
                # elif current_probability == 0:
                #     print (f"frame={frame_number} {state_from}➜{state_to}: calculated probability of zero\t" + \
                #     f"new_prob = {probability[state_to_idx,frame_number-1]} * {transition_probs[state_from][state_to]} * {gaussian_prob(evidence_vector[frame_number], emission_paras[state_to])}")
                # else:
                #     print (f"frame={frame_number} {state_from}➜{state_to}: new prob={current_probability} <= {probability[state_to_idx,frame_number]}")
            # print ("")        
        # print ("-----")        


    # print(probability)
    # print("parent")
    # print(parent_string)
    # print(parent)

    path_indexes = np.empty(evidence_count, dtype= int)
    for t in range(evidence_count-1,-1,-1):
        if t==evidence_count-1:
            path_indexes[t]=np.argmax(probability[:,t])
            # print (f"path_index[{t}]={path_indexes[0]}")
            probability = np.max(probability[:,t])
        else:
            path_indexes[t] = parent[path_indexes[t+1],t+1]
            # print (f"path element {t}={path_indexes[t]}")
    # print (f"path_indexes[{t}] = {path_indexes}")
    sequence = [states[path_indexes[t]] for t in range(evidence_count)]
    # print (f"path = {sequence}")



    print ("done 🚩🚩🚩")
    
    return sequence, probability    


########## DON'T WRITE ANY CODE OUTSIDE THE FUNCTION! ################͏󠄂͏️͏󠄌͏󠄎͏︆͏︋͏󠄏
##### CODE BELOW IS USED FOR RUNNING LOCAL TEST DON'T MODIFY IT ######͏󠄂͏️͏󠄌͏󠄎͏︆͏︋͏󠄏
tests.TestPart1b().test_viterbi_case1(part_1_a, viterbi)
tests.TestPart1b().test_viterbi_case2(part_1_a, viterbi)
tests.TestPart1b().test_viterbi_case3(part_1_a, viterbi)
tests.TestPart1b().test_viterbi_realsample1(part_1_a, viterbi)
tests.TestPart1b().test_viterbi_realsample2(part_1_a, viterbi)
tests.TestPart1b().test_viterbi_realsample3(part_1_a, viterbi)
################ END OF LOCAL TEST CODE SECTION ######################͏󠄂͏️͏󠄌͏󠄎͏︆͏︋͏󠄏

evidence_vector=[] 
states=['A1', 'A2', 'A3', 'Aend', 'N1', 'N2', 'N3', 'Nend', 'S1', 'S2', 'S3', 'Send']
prior_probs={'A1': 0.333, 'A2': 0.0, 'A3': 0.0, 'Aend': 0.0, 'N1': 0.333, 'N2': 0.0, 'N3': 0.0, 'Nend': 0.0, 'S1': 0.333, 'S2': 0.0, 'S3': 0.0, 'Send': 0.0}
transition_probs={'A1': {'A2': 0.167, 'A3': 0.0, 'A1': 0.833, 'Aend': 0.0}, 'A2': {'A1': 0.0, 'A2': 0.786, 'A3': 0.214, 'Aend': 0.0}, 'A3': {'A2': 0.0, 'A1': 0.0, 'A3': 0.727, 'Aend': 0.273}, 'Aend': {'A2': 0.0, 'A3': 0.0, 'A1': 0.0, 'Aend': 1}, 'N1': {'N3': 0.0, 'N1': 0.919, 'N2': 0.081, 'Nend': 0.0}, 'N2': {'N3': 1.0, 'N1': 0.0, 'N2': 0.0, 'Nend': 0.0}, 'N3': {'N3': 0.375, 'N2': 0.0, 'N1': 0.0, 'Nend': 0.625}, 'Nend': {'N1': 0.0, 'N2': 0.0, 'N3': 0.0, 'Nend': 1.0}, 'S1': {'S1': 0.625, 'S3': 0.0, 'S2': 0.375, 'Send': 0.0}, 'S2': {'S1': 0.0, 'S2': 0.864, 'S3': 0.136, 'Send': 0.0}, 'S3': {'S1': 0.0, 'S3': 1.0, 'S2': 0.0, 'Send': 0.0}, 'Send': {'S3': 0.0, 'S2': 0.0, 'S1': 0.0, 'Send': 1.0}}
emission_paras={'A1': (51.056, 21.986),

In [29]:
s = 'Python'
for i in range(len(s)):
    print(i, s[i])
for i in range(len(s)-1,-1,-1):
    print(i, s[i])

0 P
1 y
2 t
3 h
4 o
5 n
5 n
4 o
3 h
2 t
1 y
0 P


In [30]:
#3rd example
        # prob_ans = 8.189507039078366e-20
        # seq_ans = ['S1', 'S1', 'S1', 'S1', 'S1', 'S2', 'S2', 'S2', 'S2', 'S2', 'S2', 'S2']

In [31]:
print(np.__version__)
np.set_printoptions(suppress=False)
np.set_printoptions(threshold=np.inf)
np.set_printoptions(linewidth=np.inf)

1.26.4


In [32]:
#case 3

evidence_vector=    [20, 65, 20, 30, 45, 60, 60, 42]

states=             ['A1', 'A2', 'A3', 'Aend', 'N1', 'N2', 'N3', 'Nend', 'S1', 'S2', 'S3', 'Send']

prior_probs=        {'A1': 0.333, 'A2': 0.0, 'A3': 0.0, 'Aend': 0.0, 
                     'N1': 0.333, 'N2': 0.0, 'N3': 0.0, 'Nend': 0.0, 
                     'S1': 0.333, 'S2': 0.0, 'S3': 0.0, 'Send': 0.0}

transition_probs=   {'A1':   {'A2': 0.167, 'A3': 0.0,   'A1': 0.833, 'Aend': 0.0}, 
                     'A2':   {'A1': 0.0,   'A2': 0.786, 'A3': 0.214, 'Aend': 0.0}, 
                     'A3':   {'A2': 0.0,   'A1': 0.0,   'A3': 0.727, 'Aend': 0.273}, 
                     'Aend': {'A2': 0.0,   'A3': 0.0,   'A1': 0.0,   'Aend': 1}, 
                     'N1':   {'N3': 0.0,   'N1': 0.919, 'N2': 0.081, 'Nend': 0.0}, 
                     'N2':   {'N3': 1.0,   'N1': 0.0,   'N2': 0.0,   'Nend': 0.0}, 
                     'N3':   {'N3': 0.375, 'N2': 0.0,   'N1': 0.0,   'Nend': 0.625}, 
                     'Nend': {'N1': 0.0,   'N2': 0.0,   'N3': 0.0,   'Nend': 1.0}, 
                     'S1':   {'S1': 0.625, 'S3': 0.0,   'S2': 0.375, 'Send': 0.0}, 
                     'S2':   {'S1': 0.0,   'S2': 0.864, 'S3': 0.136, 'Send': 0.0}, 
                     'S3':   {'S1': 0.0,   'S3': 1.0,   'S2': 0.0,   'Send': 0.0}, 
                     'Send': {'S3': 0.0,   'S2': 0.0,   'S1': 0.0,   'Send': 1.0}}

emission_paras=     {'A1': (51.056, 21.986), 'A2': (28.357, 14.936), 'A3': (53.727, 16.707),  'Aend': (None, None), 
                     'N1': (38.081, 11.175), 'N2': (42.0, 2.828),    'N3': (60.0, 13.491),    'Nend': (None, None), 
                     'S1': (29.5, 8.411),    'S2': (36.182, 5.99),   'S3': (36.667, 1.886),   'Send': (None, None)}



In [33]:
#case len ==1
evidence_vector=[30] 
states=['A1', 'A2', 'A3', 'Aend', 'N1', 'N2', 'N3', 'Nend', 'S1', 'S2', 'S3', 'Send']
prior_probs={'A1': 0.333, 'A2': 0.0, 'A3': 0.0, 'Aend': 0.0, 
             'N1': 0.333, 'N2': 0.0, 'N3': 0.0, 'Nend': 0.0, 
             'S1': 0.333, 'S2': 0.0, 'S3': 0.0, 'Send': 0.0}

transition_probs={'A1': {'A2': 0.167, 'A3': 0.0, 'A1': 0.833, 'Aend': 0.0}, 
                  'A2': {'A1': 0.0, 'A2': 0.786, 'A3': 0.214, 'Aend': 0.0}, 
                  'A3': {'A2': 0.0, 'A1': 0.0, 'A3': 0.727, 'Aend': 0.273}, 
                  'Aend': {'A2': 0.0, 'A3': 0.0, 'A1': 0.0, 'Aend': 1}, 

                  'N1': {'N3': 0.0, 'N1': 0.919, 'N2': 0.081, 'Nend': 0.0}, 
                  'N2': {'N3': 1.0, 'N1': 0.0, 'N2': 0.0, 'Nend': 0.0}, 
                  'N3': {'N3': 0.375, 'N2': 0.0, 'N1': 0.0, 'Nend': 0.625}, 
                  'Nend': {'N1': 0.0, 'N2': 0.0, 'N3': 0.0, 'Nend': 1.0}, 

                  'S1': {'S1': 0.625, 'S3': 0.0, 'S2': 0.375, 'Send': 0.0}, 
                  'S2': {'S1': 0.0, 'S2': 0.864, 'S3': 0.136, 'Send': 0.0}, 
                  'S3': {'S1': 0.0, 'S3': 1.0, 'S2': 0.0, 'Send': 0.0}, 
                  'Send': {'S3': 0.0, 'S2': 0.0, 'S1': 0.0, 'Send': 1.0}}

emission_paras={'A1': (51.056, 21.986), 'A2': (28.357, 14.936), 'A3': (53.727, 16.707), 'Aend': (None, None), 
                'N1': (38.081, 11.175), 'N2': (42.0, 2.828),    'N3': (60.0, 13.491),   'Nend': (None, None), 
                'S1': (29.5, 8.411),    'S2': (36.182, 5.99),   'S3': (36.667, 1.886),  'Send': (None, None)}

### Part 2a: Multidimensional Output Probabilities
### _[Required for CS6601: 6 Points]_ _[Extra Credit for CS3600: 3 Points]_

In Part 1a, we used right-hand Y-axis coordinates as our sole feature, now we are going to use two features - the same right-hand Y-axis coordinates and the right-thumb Y-axis coordinates. Using observations from both the right hand and the right thumb as features can increase the accuracy of our model when dealing with more complex sentences.

Here you are given the transition probabilities and the means & standard deviations for emission parameters of right-thumb Y-axis locations, following the same procedure conducted in Part 1a.

<img src="part_2_a_probs_updated.png" alt="2a_probs">

ALLIGATOR | State 1 | State 2 | State 3
--- | --- | --- | --- 
Mean | 53.529 | 40.769 | 51.000
Std | 17.493 | 6.104 | 12.316

NUTS | State 1 | State 2 | State 3
--- | --- | --- | --- 
Mean | 36.318 | 60.000 | 37.476
Std | 7.376 | 15.829 | 8.245

SLEEP | State 1 | State 2 | State 3
--- | --- | --- | --- 
Mean | 35.357 | 31.462 | 38.333
Std | 7.315 | 5.048 | 7.409

#### Functions to complete:
1. `part_2_a()`


In [34]:
#export
def part_2_a():
    """Provide probabilities for the word HMMs outlined below.
    Now, at each time frame you are given 2 observations (right hand Y
    position & right thumb Y position). Use the result you derived in
    part_1_a, accompany with the provided probability for right thumb, create
    a tuple of (right-hand-y, right-thumb-y) to represent high-dimension transition & 
    emission probabilities.
    """

     # TODO: complete this function.͏󠄂͏️͏󠄌͏󠄎͏︆͏︋͏󠄏

    """Word ALLIGATOR"""
    a_prior_probs = {
        'A1': 0.333,
        'A2': 0.,
        'A3': 0.,
        'Aend': 0.
    }
    # example: {'A1': {'A1' : (right-hand Y, right-thumb Y), ... }͏󠄂͏️͏󠄌͏󠄎͏︆͏︋͏󠄏
    a_transition_probs = {
        'A1': {'A3': (0., 0.), 'A2': (0.167, 0.176), 'A1': (0.833, 0.824), 'Aend': (0., 0.)},
        'A2': {'A1': (0., 0.), 'A3': (0.214, 0.231), 'A2': (0.786, 0.769), 'Aend': (0., 0.)},
        'A3': {'A2': (0., 0.), 'A3': (0.727, 0.769), 'A1': (0., 0.), 'Aend': (0.273, 0.231)},
        'Aend': {'A1': (0., 0.), 'A2': (0., 0.), 'A3': (0., 0.), 'Aend': (1., 1.)}
    }
    # example: {'A1': [(right-hand-mean, right-thumb-std), (right-hand--mean, right-thumb-std)] ...}͏󠄂͏️͏󠄌͏󠄎͏︆͏︋͏󠄏
    a_emission_paras = {
        'A1': [(51.056, 21.986), (53.529 ,  17.493)],
        'A2': [(28.357, 14.936), (40.769,    6.104)],
        'A3': [(53.727, 16.707), (51.000,   12.316)],
        'Aend': [(None, None), (None, None)]
    }

    """Word NUTS"""
    n_prior_probs = {
        'N1': 0.333,
        'N2': 0.,
        'N3': 0.,
        'Nend': 0.
    }
    n_transition_probs = {
        'N1': {'N2': (0.081, 0.136), 'N1': (0.919, 0.864), 'N3': (0., 0.), 'Nend': (0., 0.)},
        'N2': {'N2': (0., 0.727), 'N3': (1., 0.273), 'N1': (0., 0.), 'Nend': (0., 0.)},
        'N3': {'N3': (0.375, 0.800), 'N1': (0., 0.), 'N2': (0., 0.), 'Nend': (0.625, 0.200)},
        'Nend': {'N3': (0., 0.), 'N1': (0., 0.), 'N2': (0., 0.), 'Nend': (1., 1.)}
    }
    n_emission_paras = {
        'N1': [ (38.081, 11.175), (36.318, 7.376)],
        'N2': [ (42.0, 2.828), (60,  15.829 )],
        'N3': [(60.0, 13.491),(37.476, 8.245)],
        'Nend': [(None, None), (None, None)]
    }

    """Word SLEEP"""
    s_prior_probs = {
        'S1': 0.333,
        'S2': 0.,
        'S3': 0.,
        'Send': 0.
    }
    s_transition_probs = {
        'S1': {'S3': (0., 0.), 'S2': (0.375, 0.214), 'S1': (0.625, 0.786), 'Send': (0., 0.)},
        'S2': {'S1': (0., 0.), 'S2': (0.864, 0.769), 'S3': (0.136, 0.231), 'Send': (0., 0.)},
        'S3': {'S1': (0., 0.), 'S3': (1., 0.), 'S2': (0., 0.500), 'Send': (0., 0.500)},
        'Send': {'S1': (0., 0.), 'S2': (0., 0.), 'S3': (0., 0.), 'Send': (1., 1.)}
    }
    s_emission_paras = {
        'S1': [(29.5, 8.411), (35.357, 7.315)],
        'S2': [(36.182, 5.99), (31.462 , 5.048)],
        'S3': [(36.667, 1.886), (38.333, 7.409)],
        'Send': [(None, None), (None, None)]
    }
    return (a_prior_probs, a_transition_probs, a_emission_paras,
            n_prior_probs, n_transition_probs, n_emission_paras,
            s_prior_probs, s_transition_probs, s_emission_paras)

########## DON'T WRITE ANY CODE OUTSIDE THE FUNCTION! ################͏󠄂͏️͏󠄌͏󠄎͏︆͏︋͏󠄏
##### CODE BELOW IS USED FOR RUNNING LOCAL TEST DON'T MODIFY IT ######͏󠄂͏️͏󠄌͏󠄎͏︆͏︋͏󠄏
tests.TestPart2a().test_prior(part_2_a)
tests.TestPart2a().test_a_emission(part_2_a)
tests.TestPart2a().test_n_emission(part_2_a)
tests.TestPart2a().test_s_emission(part_2_a)
tests.TestPart2a().test_a_transition(part_2_a)
tests.TestPart2a().test_n_transition(part_2_a)
tests.TestPart2a().test_s_transition(part_2_a)
################ END OF LOCAL TEST CODE SECTION ######################͏󠄂͏️͏󠄌͏󠄎͏︆͏︋͏󠄏

UnitTest test_prior passed successfully!
UnitTest test_a_emission passed successfully!
UnitTest test_n_emission passed successfully!
UnitTest test_s_emission passed successfully!
UnitTest test_a_transition passed successfully!
UnitTest test_n_transition passed successfully!
UnitTest test_s_transition passed successfully!


In [ ]:
states = ['A1', 'A2', 'A3', 'Aend', 'N1', 'N2', 'N3', 'Nend', 'S1', 'S2', 'S3', 'Send']



### Part 2b: Improving the Viterbi Trellis
### _[Required for CS6601: 39 Points]_ _[Extra Credit for CS3600: 7 Points]_

Modify the Viterbi Trellis function to allow multiple observed values (Y locations of the right hand and the right thumb) for a state. The return format should be identical to Part 1b.

Note: **DO NOT** consult any external sources other than the Wikipedia PDF in the assignment. Failure to abide by this requirement will lead to a 0 on the assignment.

#### Functions to complete:
1. `multidimensional_viterbi()`


In [83]:
#export
def multidimensional_viterbi(evidence_vector, states, prior_probs,
                             transition_probs, emission_paras):
    """Decode the most likely word phrases generated by the evidence vector.
    States, prior_probs, transition_probs, and emission_probs will now contain
    all the words from part_2_a.
    Evidence vector is a list of tuples where the first element of each tuple is the right 
    hand coordinate and the second element is the right thumb coordinate.
    """
    # print (f"evidence_vector={evidence_vector}\n \
    #             states={states}\n\
    #             prior_probs={prior_probs}\n\
    #             transition_probs={transition_probs}\n\
    #             emission_paras={emission_paras}")
    state_number = enumerate(states)
    print (f"state_number={state_number}")
    if len(evidence_vector)==0:
        sequence = []
        probability = 0.0
        return sequence, probability

    if len(evidence_vector)==1:
        mymatrix = np.array([prior_probs["A1"] * gaussian_prob(evidence_vector[0][0], emission_paras["A1"][0]) * gaussian_prob(evidence_vector[0][1], emission_paras["A1"][1]),
                             prior_probs["N1"] * gaussian_prob(evidence_vector[0][0], emission_paras["N1"][0]) * gaussian_prob(evidence_vector[0][1], emission_paras["N1"][1]),
                             prior_probs["S1"] * gaussian_prob(evidence_vector[0][0], emission_paras["S1"][0]) * gaussian_prob(evidence_vector[0][1], emission_paras["S1"][1])])
        probability = np.max(mymatrix)
        sequence = [np.array(["A1","N1","S1"])[np.argmax(mymatrix)]]
        return sequence, probability
    
    evidence_count = len(evidence_vector)
    state_count = len(states)
    assert isinstance(evidence_count,int), "evidence_count should be an int"
    assert isinstance(state_count   ,int), "state_count should be an int"
    
    probability = np.zeros([state_count  ,evidence_count],dtype=float) #  States (i) x Evidence Matrix (j). 
    parent = np.empty([state_count  ,evidence_count],dtype=int) # will store how we arrived 
    parent_string = np.empty([state_count  ,evidence_count],dtype=object) # will store how we arrived 

    #this matrix will hold the maximum probability to arrive at state
    
    
    # for i in range (state_count):
    #     for j in range (evidence_count):
    #         proability[i,j] = gaussian_prob(x=evidence_vector[j], para_tuple=emission_paras[states[i]])
        
    # print (f"prob_mat=\n{probability}")
    # prob_mat2 = np.fromfunction(lambda i, j: gaussian_prob(x=evidence_vector[j], para_tuple=emission_paras[states[i]]),\
    #                                                        shape=[state_count  ,evidence_count],\
    #                                                         dtype=np.float16) #  States x Evidence Matrix 
    # print (f"allclose= {np.allclose(prob_mat,prob_mat2)}")
    # we will go horizontally over the matrix, calculating for each step the probablilty of reaching that step
    # path =[str]
    parent_fullpath_string = np.empty([state_count  ,evidence_count],dtype=object) # will store how we arrived 

    for i in range (state_count):
        probability[i,0] = prior_probs[states[i]] * \
              gaussian_prob(evidence_vector[0][0], emission_paras[states[i]][0]) * \
              gaussian_prob(evidence_vector[0][1], emission_paras[states[i]][1])
        parent_fullpath_string[i,0]=[]
        
        # parent[i,0] = state_to_idx
        # parent_string[state_from_idx,0] = states[i]
        # path[0] = states[np.argmax(probability[:,0])]

    for frame_number in range (1,evidence_count):
        for state_from_idx in range(state_count):
            # array_to = []
            # array_to_value = []
            # for state_to in transition_probs[state_from].keys():
            #     array_to.append(state_to)
            #     current_probability = probability[state_from_idx,frame_number-1] * \
            #                           transition_probs[state_from][state_to][0] * \
            #                           transition_probs[state_from][state_to][1] * \
            #                           gaussian_prob(evidence_vector[frame_number][0], emission_paras[state_to][0]) * \
            #                           gaussian_prob(evidence_vector[frame_number][1], emission_paras[state_to][1])
            #     array_to.append(current_probability)

            # # probability = 
            # # sequence = [np.array(["A1","N1","S1"])[np.argmax(mymatrix)]]        
            # probability[state_to_idx,frame_number] = current_probability
            # parent[state_to_idx,frame_number] = state_from_idx
            # parent_string[state_to_idx,frame_number] = states[state_from_idx]

            for state_to_idx in range(state_from_idx,state_count):

                state_from , state_to = states[state_from_idx] , states[state_to_idx]

                if not state_to in transition_probs[state_from]:
                    # print (f"frame={frame_number} {state_from}➜{state_to}: skipping because no connection")
                    break
                if 0 == transition_probs[state_from][state_to]:
                    # print (f"frame={frame_number} {state_from}➜{state_to}: skipping because chance is zero")
                    break #if zero probability == no need to consider this one
                
                # current probablity is the chance of getting here via the path we got here, which is a multiplication of how we got here 
                current_probability = probability[state_from_idx,frame_number-1] * \
                                      transition_probs[state_from][state_to][0] * \
                                      transition_probs[state_from][state_to][1] * \
                                      gaussian_prob(evidence_vector[frame_number][0], emission_paras[state_to][0]) * \
                                      gaussian_prob(evidence_vector[frame_number][1], emission_paras[state_to][1])
                # print (f"⭐new_prob {current_probability}= {probability[state_from_idx,frame_number-1]} * {transition_probs[state_from][state_to]} * {gaussian_prob(evidence_vector[frame_number], emission_paras[state_to])}")

                if current_probability > probability[state_to_idx,frame_number] :
                    #if we are here, then a new maximum occured and the new parent is 'state_from' (index 'state_from_idx')
                    # print (f"frame={frame_number} {state_from}➜{state_to}: 💾 Updating: new prob={current_probability} > {probability[state_to_idx,frame_number]} New Parent: {states[state_to_idx]}")
                    probability[state_to_idx,frame_number] = current_probability
                    parent[state_to_idx,frame_number] = state_from_idx
                    parent_string[state_to_idx,frame_number] = states[state_from_idx]
                    if parent_fullpath_string[state_to_idx,frame_number-1]  is None:
                        parent_fullpath_string[state_to_idx,frame_number] = [state_from_idx]
                    else:
                        parent_fullpath_string[state_to_idx,frame_number] = []
                        parent_fullpath_string[state_to_idx,frame_number] += parent_fullpath_string[state_from_idx,frame_number-1]
                        parent_fullpath_string[state_to_idx,frame_number] += [state_from]
                        
                # elif current_probability == 0:
                #     print (f"frame={frame_number} {state_from}➜{state_to}: calculated probability of zero\t" + \
                #     f"new_prob = {probability[state_to_idx,frame_number-1]} * {transition_probs[state_from][state_to]} * {gaussian_prob(evidence_vector[frame_number], emission_paras[state_to])}")
                # else:
                #     print (f"frame={frame_number} {state_from}➜{state_to}: new prob={current_probability} <= {probability[state_to_idx,frame_number]}")
            # print ("")        
        # print ("-----")        


    # print(probability)
    # print("parent")
    # print(parent_string)
    # print(parent)

    ending_state_index = np.argmax(probability[:,evidence_count-1])
    sequence2 = parent_fullpath_string[ending_state_index,evidence_count-1]
    sequence2 += [states[ending_state_index]]
    print (f"seq2 type = {type(sequence2)}")
    return_probability = np.max(probability[:,evidence_count-1])
    # path_indexes = np.empty(evidence_count, dtype= int)
    # for t in range(evidence_count-1,-1,-1):
    #     if t==evidence_count-1:
    #         path_indexes[t]=np.argmax(probability[:,t])
    #         # print (f"path_index[{t}]={path_indexes[0]}")
    #         probability = np.max(probability[:,t])
    #         # print(f"best path ends in cell = {evidence_count-1},{np.argmax(probability[:,t])}")
    #     else:
    #         path_indexes[t] = parent[path_indexes[t+1],t+1]
    #         # print (f"path element {t}={path_indexes[t]}")
    # # print (f"path_indexes[{t}] = {path_indexes}")
    # sequence = [states[path_indexes[t]] for t in range(evidence_count)]
    # print(f"done💚  path = {sequence}")
    print(f"done💚  path = {sequence2}")
    # print (f"{sequence==sequence2}")
    # print(f"best ending index: {np.argmax(probability[:,evidence_count-1])}")
    # print(f"best ending path: {parent_fullpath_string[np.argmax(probability[:,evidence_count-1]),evidence_count-1]}")
    return sequence2, return_probability    

########## DON'T WRITE ANY CODE OUTSIDE THE FUNCTION! ################͏󠄂͏️͏󠄌͏󠄎͏︆͏︋͏󠄏
##### CODE BELOW IS USED FOR RUNNING LOCAL TEST DON'T MODIFY IT ######͏󠄂͏️͏󠄌͏󠄎͏︆͏︋͏󠄏
tests.TestPart2b().test_viterbi_case1(part_2_a, multidimensional_viterbi)
tests.TestPart2b().test_viterbi_case2(part_2_a, multidimensional_viterbi)
tests.TestPart2b().test_viterbi_case3(part_2_a, multidimensional_viterbi)
tests.TestPart2b().test_viterbi_realsample1(part_2_a, multidimensional_viterbi)
tests.TestPart2b().test_viterbi_realsample2(part_2_a, multidimensional_viterbi)
tests.TestPart2b().test_viterbi_realsample3(part_2_a, multidimensional_viterbi)
################ END OF LOCAL TEST CODE SECTION ######################͏󠄂͏️͏󠄌͏󠄎͏︆͏︋͏󠄏

UnitTest test_viterbi_case1 passed successfully!
UnitTest test_viterbi_case2 passed successfully!
UnitTest test_viterbi_case3 passed successfully!
seq2 type = <class 'list'>
done💚  path = ['A1', 'A1', 'A2', 'A2', 'A2', 'A3', 'A3', 'A3']
UnitTest test_viterbi_realsample1 passed successfully!
seq2 type = <class 'list'>
done💚  path = ['N1', 'N1', 'N1', 'N1', 'N1', 'N1', 'N1', 'N1', 'N1']
UnitTest test_viterbi_realsample2 passed successfully!
seq2 type = <class 'list'>
done💚  path = ['S1', 'S1', 'S1', 'S1', 'S1', 'S2', 'S2', 'S2', 'S2', 'S2', 'S2', 'S2']
UnitTest test_viterbi_realsample3 passed successfully!


In [ ]:
# test_viterbi_realsample1
path = ['A1', 'A1', 'A2', 'A2', 'A2', 'A3', 'A3', 'A3']

In [36]:
#export
def return_your_name():
    """Return your name
    """
    return "Ben Hizak"

**CONGRATULATIONS!**  You have just completed your final assignment for CS6601 Artificial Intelligence.
